In [1]:
import pandas as pd
import polars as pl
import duckdb as db

In [2]:
import numpy as np

# Pandas

In [3]:
df = pd.read_parquet('../AwesomeDAG/dags/data/imdb_data.parquet')

In [4]:
df_exp = df.explode(column=['actor']).explode('writers')

In [5]:
%%timeit
# дешевая операция
df_exp.groupby('writers')['averageRating'].agg(['mean','median'])

132 ms ± 6.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%time
# огромный групбай
df_exp.groupby(['actor','directors','writers']).agg({'averageRating':'mean','primaryTitle':['unique','count']})

CPU times: user 26.7 s, sys: 88.7 ms, total: 26.8 s
Wall time: 26.8 s


averageRating  \
                                                                                  mean   
actor                  directors                writers                                  
'Cousin Brucie' Morrow Fred Barzyk              David Odell                        7.2   
                                                David R. Loxton                    7.2   
                                                Fred Barzyk                        7.2   
                                                Kurt Vonnegut Jr.                  7.2   
'Ducky' Louie          Phil Karlson             Agnes Christine Johnston           6.1   
...                                                                                ...   
Þórhallur Sigurðsson   Þórhildur Þorleifsdóttir Guðný Halldórsdóttir               7.2   
Þórhallur Sverrisson   Róbert I. Douglas        Róbert I. Douglas                  6.9   
Þórir Waagfjörð        Michael Chapman          Dale Herd                          5.2   
                                                Michael Chapman                    5.2   
                                                Paul R. Gurian                     5.2   

                                                                                         primaryTitle  \
                                                                                               unique   
actor                  directors                writers                                                 
'Cousin Brucie' Morrow Fred Barzyk              David Odell               [Between Time and Timbuktu]   
                                                David R. Loxton           [Between Time and Timbuktu]   
                                                Fred Barzyk               [Between Time and Timbuktu]   
                                                Kurt Vonnegut Jr.         [Between Time and Timbuktu]   
'Ducky' Louie          Phil Karlson             Agnes Christine Johnston                 [Black Gold]   
...                                                                                               ...   
Þórhallur Sigurðsson   Þórhildur Þorleifsdóttir Guðný Halldórsdóttir              [Stella on Holiday]   
Þórhallur Sverrisson   Róbert I. Douglas        Róbert I. Douglas               [The Icelandic Dream]   
Þórir Waagfjörð        Michael Chapman          Dale Herd                          [The Viking Sagas]   
                                                Michael Chapman                    [The Viking Sagas]   
                                                Paul R. Gurian                     [The Viking Sagas]   

                                                                                
                                                                         count  
actor                  directors                writers                         
'Cousin Brucie' Morrow Fred Barzyk              David Odell                  1  
                                                David R. Loxton              1  
                                                Fred Barzyk                  1  
                                                Kurt Vonnegut Jr.            1  
'Ducky' Louie          Phil Karlson             Agnes Christine Johnston     1  
...                                                                        ...  
Þórhallur Sigurðsson   Þórhildur Þorleifsdóttir Guðný Halldórsdóttir         1  
Þórhallur Sverrisson   Róbert I. Douglas        Róbert I. Douglas            1  
Þórir Waagfjörð        Michael Chapman          Dale Herd                    1  
                                                Michael Chapman              1  
                                                Paul R. Gurian               1  

[578898 rows x 3 columns]

In [7]:
# Простая 126 ms
# Сложная 25.7 s

In [8]:
pandas_result = {'easy': 126,'hard':25.7 * 1000}

# Polars

In [9]:
df_pl = pl.read_parquet('../AwesomeDAG/dags/data/imdb_data.parquet')

In [10]:
df_pl_exp = df_pl.explode('actor').explode('writers')

In [11]:
%%timeit
# дешевая операция
df_pl_exp.groupby('writers').agg([pl.col('averageRating').mean().alias('mean'),pl.col('averageRating').median().alias('median')])

20.4 ms ± 625 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%time
# огромный групбай
df_pl_exp.groupby(['actor','directors','writers']).agg([pl.col('averageRating').mean(),
                                                        pl.col('primaryTitle').unique(),
                                                        pl.col('primaryTitle').count().alias('count_titles')])

CPU times: user 1.41 s, sys: 178 ms, total: 1.59 s
Wall time: 492 ms


actor,directors,writers,averageRating,primaryTitle,count_titles
str,str,str,f64,list[str],u32
"""Gary Cooper""","""Sam Wood""","""Vincent Lawren...",7.6,"[""The Pride of the Yankees""]",1
"""John Carradine...","""Ted V. Mikels""","""Wayne Rogers""",3.1,"[""The Astro-Zombies""]",1
"""Ray Danton""","""Joseph L. Scan...","""Hal Fimberg""",4.8,"[""Our Man Flint: Dead on Target""]",1
"""Michael J. And...","""Vojtech Jasný""","""David Sigmund""",5.1,"[""The Great Land of Small""]",1
"""Larry Brandenb...","""George Miller""","""George Malko""",3.9,"[""Tidal Wave: No Escape""]",1
"""Dilip Kumar""","""Nitin Bose""","""Azm Bazidpuri""",7.1,"[""Deedar""]",1
"""Nesbitt Blaisd...",null,"""Barak Goodman""",7.5,"[""Scottsboro: An American Tragedy""]",1
"""Jeff Bridges""",null,"""Keith Fulton""",7.3,"[""Lost in La Mancha""]",1
"""Mikolaj Grabow...","""Lukasz Karwows...","""Lukasz Karwows...",5.4,"[""Expecting Love""]",1


In [13]:
# Простая 21 ms
# Сложная 512.7 ms

In [14]:
polars_result = {'easy': 21,'hard':512}

# duckdb

In [15]:
conn = db.connect()
df_db = conn.execute('''
                    create temp table data
                    as
                    select 
                    *
                    from read_parquet('../AwesomeDAG/dags/data/imdb_data.parquet')
                    ''')
conn.close()

In [16]:
conn = db.connect()
df_db = conn.execute('''
                    create temp table my_data as select * from df_exp
                    ''')

In [ ]:
# NO IO

In [17]:
%%timeit
# простая операция без IO в датафрейм/ чисто расчет
df_db.query('''

            select 
            writers,
            avg(averageRating) as mean,
            median(averageRating) as median
            from my_data
            group by 1

            ''')

37.3 µs ± 552 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [18]:
%%time
df_db.query('''

            select 
            actor,
            directors,
            writers,
            avg(averageRating) as mean,
            list(distinct primaryTitle) as uniq,
            count(primaryTitle) as cnt
            from my_data
            group by 1,2,3

            ''')

CPU times: user 260 µs, sys: 0 ns, total: 260 µs
Wall time: 266 µs


---------------------
--- Relation Tree ---
---------------------
Subquery

---------------------
-- Result Columns  --
---------------------
- actor (VARCHAR)
- directors (VARCHAR)
- writers (VARCHAR)
- mean (DOUBLE)
- uniq (VARCHAR[])
- cnt (BIGINT)

---------------------
-- Result Preview  --
---------------------
actor	directors	writers	mean	uniq	cnt	
VARCHAR	VARCHAR	VARCHAR	DOUBLE	VARCHAR[]	BIGINT	
[ Rows: 10]
William Courtenay	Alexander Black	Alexander Black	5.3	[Miss Jerry]	1
Chauncey Depew	Alexander Black	Alexander Black	5.3	[Miss Jerry]	1
NULL	Enoch J. Rector	NULL	5.3	[The Corbett-Fitzsimmons Fight]	1
John Tait	Charles Tait	Charles Tait	6.0	[The Story of the Kelly Gang]	1
Nicholas Brierley	Charles Tait	Charles Tait	6.0	[The Story of the Kelly Gang]	1
Norman Campbell	Charles Tait	Charles Tait	6.0	[The Story of the Kelly Gang]	1
Valdemar Psilander	Urban Gad	Urban Gad	5.8	[Den sorte drøm]	1
Valdemar Psilander	Urban Gad	Gebhard Schätzler-Perasini	5.8	[Den sorte drøm]	1
Gunnar Hels

In [ ]:
# IO

In [19]:
%%timeit
# простая операция с IO в датафрейм
df_db.query('''

            select 
            writers,
            avg(averageRating) as mean,
            median(averageRating) as median
            from my_data
            group by 1

            ''').df()

55.7 ms ± 830 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%%time
# сложная операция с IO в датафрейм
df_db.query('''

            select 
            actor,
            directors,
            writers,
            avg(averageRating) as mean,
            list(distinct primaryTitle) as uniq,
            count(primaryTitle) as cnt
            from my_data
            group by 1,2,3

            ''').df()

CPU times: user 2.27 s, sys: 165 ms, total: 2.43 s
Wall time: 1.89 s


,actor,directors,writers,mean,uniq,cnt
0,William Courtenay,Alexander Black,Alexander Black,5.3,[Miss Jerry],1
1,Chauncey Depew,Alexander Black,Alexander Black,5.3,[Miss Jerry],1
2,NaN,Enoch J. Rector,NaN,5.3,[The Corbett-Fitzsimmons Fight],1
3,John Tait,Charles Tait,Charles Tait,6.0,[The Story of the Kelly Gang],1
4,Nicholas Brierley,Charles Tait,Charles Tait,6.0,[The Story of the Kelly Gang],1
...,...,...,...,...,...,...
688137,Yoon Jong-Bin,Kim Min-Kyung,Ji-Young Oh,7.3,[Do Do Sol Sol La La Sol],1
688138,Norbert Leo Butz,NaN,Ashley Michel Hoban,6.3,[The Girl from Plainville],1
688139,Luca Argentero,Riccardo Milani,Giulia Calenda,5.8,[Like a Cat on a Highway 2],1
688140,David Giuntoli,Sam Liu,Denny O'Neil,6.1,[Batman: Soul of the Dragon],1


In [21]:
%%time
# взять и агрегировать готовый сет
with db.connect() as cn:
    cn.query('''

            select 
            actor,
            directors,
            writers,
            avg(averageRating) as mean,
            list(distinct primaryTitle) as uniq,
            count(primaryTitle) as cnt
            from df_exp
            group by 1,2,3

            ''').df()

CPU times: user 1.86 s, sys: 321 ms, total: 2.18 s
Wall time: 2.09 s


In [22]:
# IO
# Простая 55.6 ms
# Сложная 1.77 s
# Сложная прям из существующего df 2.07 s

# NO IO 
# Простая 37.8 us
# Сложная 236  us


In [23]:
db_io_result = {'easy': 55,'hard':1.7 * 1000}
db_no_io_result = {'easy': 37.8 / 1000,'hard':236 / 1000}

# Итого

In [24]:
pd.DataFrame([
    pandas_result,
    polars_result,
    db_io_result,
    db_no_io_result
],index=['pandas','polars','duckdb_io','duckdb_compute'])

,easy,hard
pandas,126.0000,25700.000
polars,21.0000,512.000
duckdb_io,55.0000,1700.000
duckdb_compute,0.0378,0.236
